In [1]:
import pandas as pd
import numpy as np

# Función de Predicción

$h(x_i) = x_i \cdot w^t $

$h(X) = XW^t$ 

In [ ]:
def h(X, W):
    return np.dot(X, W)

# Loss Function Lasso

$\mathcal{L} = ||y - h(X)||_2^2 + \lambda ||W||_1$

$\mathcal{L} = \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 + \lambda \sum_{j=0}^{p} |w_j|$

In [ ]:
def loss_lasso(Y, Y_aprox, W, lam):
    return np.sum((Y - Y_aprox)**2) + lam * np.sum(abs(W))

## Derivadas Lasso

$\frac{\partial \mathcal{L}}{\partial W_{ij}} = \lambda \cdot \text{sign}(W_{ij}) $

In [ ]:
def derivates_lasso(Y, Y_aprox, X, lam, W):
    return 2 * np.dot((Y - Y_aprox).T, X) + lam * np.sum(W/abs(W))

# Actualización pesos

In [ ]:
def update(W, alpha, dW):
    return W - alpha * dW

# Training

In [ ]:
def train(X, Y, epochs, alfa,lamda):
    W = np.random.randn(X.shape[1], 1) * 0.01
    loss = []
    for i in range(epochs):
        Y_aprox = h(X, W)
        dW = derivates_lasso(Y, Y_aprox, X)
        W = update(W,alfa, dW)
        L = loss_lasso(Y, Y_aprox)
        loss.append(L)
        if ((i%1000)==0):
          print("loss value error :" + str(L))
    return W, loss